In [1]:
import os
import sys
import glob
import pathlib
import numpy as np
import pandas as pd
# import pylab as plt

from lightgbm import LGBMClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error as MAE

In [2]:
sys.path.append(os.path.abspath('../src/'))
    
from model_selection import cross_val_score

from data import data_generator

In [3]:
from quality_estimator import BaseQualityEstimator

### Check metrics:

In [4]:
X, Xy, y = data_generator(root_dir="../data/decomposed/")

In [10]:
q_clf = BaseQualityEstimator(metrics=["dice_coefficient", 
#                                       "mae", "mse", 
#                                       "hausdorff_distance", 
#                                       "surface_distances",
#                                       "assd"
                                     ], 
                             unary_metrics=["area"],
                             matching_metrics=["match2tpr"],
                             meta_clf=LGBMClassifier(max_depth=3))

In [11]:
cross_val_score(q_clf, X, Xy, y, cv=10, scoring=MAE, random_state=4)

[0.5555555555555556,
 0.6111111111111112,
 0.7777777777777778,
 0.5555555555555556,
 0.7777777777777778,
 0.8333333333333334,
 0.6111111111111112,
 0.8888888888888888,
 0.5555555555555556,
 0.5]

In [14]:
np.mean(_8)

0.661111111111111

In [16]:
y_dummy = np.array([3 for i in range(len(y))])

In [17]:
MAE(y, y_dummy)

1.1777777777777778